In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns


from scipy import stats
from scipy.stats import norm, skew

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings('ignore')

In [ ]:
arabica_data = pd.read_csv('../input/coffee-quality-database-from-cqi/arabica_data_cleaned.csv')
arabica_data.head()

# Washed / Wet processing method
- Variety null -> other
- Coffee_Name Create
- Rating column create by tastes' score's -> it will be encoded by 'get_dummies'

In [ ]:
arabica_data_wet = arabica_data.loc[arabica_data['Processing.Method'] == 'Washed / Wet']

# Binding 'Other' in 'Variety' is None cases
arabica_data_wet['Variety'] = arabica_data_wet['Variety'].fillna('Other')

arabica_data_wet['Coffee_Name'] = [arabica_data_wet['Country.of.Origin'].iloc[i] + '_' + arabica_data_wet['Variety'].iloc[i]
                                     if arabica_data_wet['Variety'].iloc[i] != 'Other'
                                     else arabica_data_wet['Country.of.Origin'].iloc[i]
                                     for i in range(len(arabica_data_wet))]

In [ ]:
# Rating
tastes = ['Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance']
uniformity_sweetness = ['Uniformity', 'Sweetness']


for i in range(len(tastes)):
    arabica_data_wet['{}_Rating'.format(tastes[i])] = 0

for i in range(len(arabica_data_wet)):
    ratings = arabica_data_wet[tastes].iloc[i].sort_values(ascending = False).index
    for rating in range(len(ratings)):
        arabica_data_wet['{}_Rating'.format(ratings[rating])].iloc[i] = rating + 1

In [ ]:
### Ordering

"""arabica_data_wet['Ordering'] = 0
Ordering = set()

features = tastes + uniformity_sweetnees + ['Ordering']
for column in arabica_data_wet.columns:
    if column in features:
        coffee[column] = arabica_data_wet[column]
"""

"""
# categorical feature : 더미변수후 cluster_data와 합침
Order_dummies =  pd.concat((coffee['Coffee_Name'], pd.get_dummies(coffee['Ordering'])), axis = 1).groupby('Coffee_Name').max().reset_index()
cluster_data = pd.concat((cluster_data, Order_dummies.drop(['Coffee_Name'], axis = 1)), axis = 1)
cluster_data.shape
"""

In [ ]:
# numerical features : adding mean value by tastes' columns
cluster_data = pd.DataFrame({'Coffee_Name' : arabica_data_wet.groupby('Coffee_Name')['Aroma'].mean().index})
ratings = ['Aroma_Rating', 'Flavor_Rating', 'Aftertaste_Rating', 'Acidity_Rating', 'Body_Rating', 'Balance_Rating']
features = tastes + uniformity_sweetness + ratings
for column in features:
    cluster_data[column] = arabica_data_wet.groupby('Coffee_Name')[column].mean().values
    

In [ ]:
model = TSNE(n_components = 2, random_state = 0, perplexity = 50)
tsne = model.fit_transform(cluster_data.drop(['Coffee_Name'], axis = 1).values)

std = StandardScaler()
s = std.fit_transform(cluster_data.drop(['Coffee_Name'], axis = 1))

pca = PCA(n_components = 7)
pca.fit(cluster_data.drop(['Coffee_Name'], axis = 1))
pc = pca.transform(cluster_data.drop(['Coffee_Name'], axis = 1))
kmeans = KMeans(n_clusters = 35)
kmeans.fit(pc)

fr = pd.DataFrame({'tsne1' : tsne[:,0], 'tsne2' : tsne[:, 1], 'cluster' : kmeans.labels_})
#sns.lmplot(data = fr, x = 'tsne1', y = 'tsne2', hue = 'cluster', fit_reg = False)
print(np.sum(pca.explained_variance_ratio_))

cluster_data['Cluster'] = kmeans.labels_

# Checking Scatter

In [ ]:
"""def scatter_plot(data, x, y, hue = 'Cluster'):
    sns.lmplot(data = data, x = x, y = y, hue = hue, fit_reg = False)
    plt.show()
    
scatter_plot(cluster_data, 'Aroma', 'Acidity')"""

## Checking correlationship with each input variables by Clusters.

In [ ]:
result = pd.concat((cluster_data, pd.get_dummies(cluster_data['Cluster'])), axis = 1)
result.drop(['Cluster'],  axis = 1, inplace = True)

fig, ax = plt.subplots(figsize = (25, 20))

sns.heatmap(result.corr(), annot = True)

### I don't know why 'sns.lmplot' can't work in kaggle notebook. However it is not big deal to understand this notebook process. So I will pass that step.